In [ ]:
# Instalación de librerías:

%pip install pandas
%pip install requests
%pip install psycopg2
%pip install datetime

In [95]:
# Importación de librerías a utilizar:

import os
import datetime
import json
import pandas as pd
import requests as req
import psycopg2 as pg2
from psycopg2.extras import execute_values

In [96]:
# Seteo de variables presentes en el archivo .env:

CLAVE_API = os.environ.get("API_KEY")
USUARIO_BD = os.environ.get("DB_USER")
CONTRASENA_BD = os.environ.get("DB_PASSWORD")
HOST_BD = os.environ.get("DB_HOST")
PUERTO_BD = os.environ.get("DB_PORT")
NOMBRE_BD = os.environ.get("DB_NAME")

with open('variables.json') as json_file:
        variables_json = json.load(json_file)

nombres = variables_json["Nombres_Empresas"]
columnas_df = variables_json["Columnas_DataFrame"]
dict_aws = variables_json["Columnas_Redshift"]
list_aws = list(variables_json["Columnas_Redshift"].keys())

In [97]:
# Configuración para mostrar la totalidad de registros y campos

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [98]:
# Extracción desde la API de los datos diarios de "Los 7 Magníficos" (Alphabet, Amazon, Apple, Meta, Microsoft, Nvidia y Tesla) correspondiente al NASDAQ tomando en cuenta el último año (260 días sin contar Sabados y Domingos):

api = 'https://api.twelvedata.com/time_series?symbol=GOOGL,AMZN,AAPL,META,MSFT,NVDA,TSLA&exchange=NASDAQ&interval=1day&format=JSON&outputsize=260&apikey=' + CLAVE_API
peticion = req.get(api)
peticion

<Response [200]>

In [99]:
# Conversión de datos a formato JSON:

datos_json = peticion.json()

In [100]:
# Normalización de datos y agregado de columnas:

DataFrame = pd.DataFrame()
claves = list(datos_json.keys())
contador = 0

for empresa in datos_json:
    tabla = pd.json_normalize(datos_json[empresa]['values'])
    tabla['Codigo'] = claves[contador]
    tabla['Empresa'] = nombres[claves[contador]]
    DataFrame = pd.concat([DataFrame, tabla], axis = 0)
    contador += 1

DataFrame.columns = columnas_df

DataFrame = DataFrame.sort_values(by = ['Dia', 'Empresa'], ascending = [False, True],ignore_index = True)
DataFrame['Clave_Compuesta'] = DataFrame.Codigo.str.cat(DataFrame.Dia)
DataFrame['Columna_Temporal'] = datetime.datetime.now()
DataFrame

,Dia,Apertura,Maximo,Minimo,Cierre,Volumen,Codigo,Empresa,Clave_Compuesta,Columna_Temporal
0,2024-03-18,148.66000,152.14999,148.64000,150.30000,40401821,GOOGL,Alphabet Inc.,GOOGL2024-03-18,2024-03-18 12:52:10.668917
1,2024-03-18,175.86800,176.69000,174.74001,175.09540,12574056,AMZN,"Amazon.com, Inc.",AMZN2024-03-18,2024-03-18 12:52:10.668917
2,2024-03-18,175.60001,177.71001,174.89059,176.03619,34040987,AAPL,Apple Inc.,AAPL2024-03-18,2024-03-18 12:52:10.668917
3,2024-03-18,489.16501,493.53900,486.82001,491.76999,4650916,META,"Meta Platforms, Inc.",META2024-03-18,2024-03-18 12:52:10.668917
4,2024-03-18,414.23001,420.73001,413.79001,418.26779,6167542,MSFT,Microsoft Corporation,MSFT2024-03-18,2024-03-18 12:52:10.668917
...,...,...,...,...,...,...,...,...,...,...
1815,2023-03-07,153.70000,154.03000,151.13000,151.60001,56182000,AAPL,Apple Inc.,AAPL2023-03-07,2024-03-18 12:52:10.668917
1816,2023-03-07,189.00000,190.36000,184.00999,184.50999,36701500,META,"Meta Platforms, Inc.",META2023-03-07,2024-03-18 12:52:10.668917
1817,2023-03-07,256.29999,257.69000,253.39000,254.14999,21473180,MSFT,Microsoft Corporation,MSFT2023-03-07,2024-03-18 12:52:10.668917
1818,2023-03-07,236.00000,241.25000,232.41000,232.88000,51515388,NVDA,NVIDIA Corporation,NVDA2023-03-07,2024-03-18 12:52:10.668917


In [101]:
# Conexión a la base de datos en Amazon Redshift y creación del cursor:

try:
    conexion = pg2.connect(host = HOST_BD, port = PUERTO_BD, dbname = NOMBRE_BD, user = USUARIO_BD, password = CONTRASENA_BD)
    print('Conexión exitosa a la base de datos.')
except Exception as e:
    print('Conexión fallida a la base de datos.')
    print(e)

cursor = conexion.cursor()

Conexión exitosa a la base de datos.


In [103]:
# Creación de las tablas principal y staging en Amazon Redshift:

columnas_query_create = ''
contador_create = 1

for i in dict_aws:
        if contador_create != len(dict_aws):
                columnas_query_create = columnas_query_create + i + ' ' + dict_aws[i] + ','
        else:
                columnas_query_create = columnas_query_create + i + ' ' + dict_aws[i]
        contador_create += 1

query_create = 'CREATE TABLE IF NOT EXISTS b_arganaraz_londero_coderhouse.cotizacion_magnificos(' + columnas_query_create + ');' + '''

CREATE TABLE IF NOT EXISTS b_arganaraz_londero_coderhouse.cotizacion_magnificos_staging(''' + columnas_query_create + ');'

cursor.execute(query_create)
conexion.commit()

In [104]:
# Insercion de datos a la tabla staging creada en Amazon Redshift:

columnas_query_insert = ''
contador_insert = 1

for x in dict_aws:
        if contador_insert == 1:
                columnas_query_insert = '(' + columnas_query_insert + x + ', '
        elif contador_insert != len(dict_aws):
                columnas_query_insert = columnas_query_insert + x + ', '
        else:
                columnas_query_insert = columnas_query_insert + x + ')'
        contador_insert += 1

query_insert = 'INSERT INTO b_arganaraz_londero_coderhouse.cotizacion_magnificos_staging' + columnas_query_insert + ' VALUES %s;'

valores = [tuple(var) for var in DataFrame[list_aws].to_numpy()]
execute_values(cursor, query_insert, valores)
conexion.commit()

In [105]:
# Actualización incremental de la tabla principal en base a los datos de la tabla staging en Amazon Redshift:

query_incremental = '''
DELETE FROM b_arganaraz_londero_coderhouse.cotizacion_magnificos 
USING b_arganaraz_londero_coderhouse.cotizacion_magnificos_staging 
WHERE b_arganaraz_londero_coderhouse.cotizacion_magnificos.Clave_Compuesta = b_arganaraz_londero_coderhouse.cotizacion_magnificos_staging.Clave_Compuesta;

INSERT INTO b_arganaraz_londero_coderhouse.cotizacion_magnificos SELECT * FROM b_arganaraz_londero_coderhouse.cotizacion_magnificos_staging;

DROP TABLE b_arganaraz_londero_coderhouse.cotizacion_magnificos_staging;
'''

cursor.execute(query_incremental)
conexion.commit()

In [106]:
# Seleccion de los datos insertados a la tabla creada en Amazon Redshift para comprobación de carga correcta:

query_seleccion = '''
        SELECT * FROM b_arganaraz_londero_coderhouse.cotizacion_magnificos
        '''

cursor.execute(query_seleccion)
conexion.commit()
resultados = cursor.fetchall()

resultados = pd.DataFrame(resultados)
resultados.columns = list_aws
resultados.set_index('Clave_Compuesta', inplace = True)
resultados

,Dia,Empresa,Codigo,Apertura,Maximo,Minimo,Cierre,Volumen,Columna_Temporal
Clave_Compuesta,,,,,,,,,
AAPL2023-03-07,2023-03-07,Apple Inc.,AAPL,153.70,154.03,151.13,151.60,56182000,2024-03-18 12:52:10.668917
AAPL2023-03-10,2023-03-10,Apple Inc.,AAPL,150.21,150.94,147.61,148.50,68572400,2024-03-18 12:52:10.668917
AAPL2023-03-20,2023-03-20,Apple Inc.,AAPL,155.07,157.82,154.15,157.40,73641400,2024-03-18 12:52:10.668917
AAPL2023-03-21,2023-03-21,Apple Inc.,AAPL,157.32,159.40,156.54,159.28,73938300,2024-03-18 12:52:10.668917
AAPL2023-03-22,2023-03-22,Apple Inc.,AAPL,159.30,162.14,157.81,157.83,75701800,2024-03-18 12:52:10.668917
...,...,...,...,...,...,...,...,...,...
TSLA2024-03-01,2024-03-01,"Tesla, Inc.",TSLA,200.52,204.52,198.50,202.64,82099200,2024-03-18 12:52:10.668917
TSLA2024-03-04,2024-03-04,"Tesla, Inc.",TSLA,198.73,199.75,186.72,188.14,134334900,2024-03-18 12:52:10.668917
TSLA2024-03-06,2024-03-06,"Tesla, Inc.",TSLA,179.99,181.58,173.70,176.54,107920900,2024-03-18 12:52:10.668917


In [107]:
# Cierre de cursor y conexión:

cursor.close()
conexion.close()